In [24]:
from elasticsearch import Elasticsearch, helpers
from pymongo import MongoClient
import json
import pprint

In [25]:
es = Elasticsearch(HOST="http://localhost", PORT=9200,timeout=60)
client=MongoClient()
db_gs=client["scholar_antioquia"]
gs_cites_col=db_gs["stage_cites"]

In [26]:
if es.indices.exists(index="test"):
    es.indices.delete(index="test")

In [27]:
if not es.indices.exists(index="test"):
    es.indices.create(index="test")

In [14]:
cites=list(gs_cites_col.find().limit(100))

# Insertamos de a uno

In [28]:
for i in cites:
    title=i['cite']['title']
    _id=str(i['_id'])
    body={'title':title}
    es.index(index="test", doc_type="text", id=_id,body=body )
    

In [21]:
title=cites[5]['cite']['title'].lower()
title

'trastornos de la coagulación en la unidad de cuidados intensivos pediátricos'

In [31]:
%%time
body = {
    "from":0,
    "size":2,
    "query": {
        "match": {
            "PaperTitle":title
        }
    }
}

res = es.search(index="mag", body=body)
res

CPU times: user 2.92 ms, sys: 0 ns, total: 2.92 ms
Wall time: 1.84 s


{'took': 1831,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 35.615406,
  'hits': [{'_index': 'mag',
    '_type': '_doc',
    '_id': '2799480043',
    '_score': 35.615406,
    '_source': {'PaperTitle': 'an algorithm for preclinical diagnosis of alzheimer s disease'}},
   {'_index': 'mag',
    '_type': '_doc',
    '_id': '2472424722',
    '_score': 26.207619,
    '_source': {'PaperTitle': 'preclinical diagnosis of ischemic heart disease'}}]}}

# Insertamos en Bulk

In [23]:
es_entries = []
for i in cites:
    title=i['cite']['title']
    _id=str(i['_id'])    
    entry = {"_index": "test",
             "_type": "text", 
             "_id":_id,
             "_source": {'title':title} }

    es_entries.append(entry)    
try:
    helpers.bulk(es, es_entries, refresh=True, request_timeout=60) 
except Exception as e:
    # This can happen if the server is restarted or the connection becomes unavilable
    print(str(e))

/home/colav/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)
